This program collects the corpus of school books on Geometry from https://docbaza.ru.

In [1]:
import urllib.request
import re
import os
import shutil
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'

In [5]:
common_link = "https://docbaza.ru"
links = ["/urok/geometr/07/001/", "/urok/geometr/07/003/", "/urok/geometr/07/005/", "/urok/geometr/07/006/", "/urok/geometr/10/001/", "/urok/geometr/10/004/", "/urok/geometr/10/005/", "/urok/geometr/10/007/", ]

In [12]:
def download_html(pageUrl): ## downloading text of an html page
    try:
        page = urllib.request.Request(pageUrl, headers={'User-Agent':user_agent})
        with urllib.request.urlopen(page) as hpage:
            html = hpage.read().decode('windows-1251')
        return html
    except:
        print('there\'s no such page', pageUrl)
        html = None
        return html

In [50]:
#test
mainpage = download_html('https://docbaza.ru/urok/geometr/07/001/')
total = get_numbofpages(mainpage)
print(total)

199


In [65]:
##test
page = download_html('https://docbaza.ru/urok/geometr/07/001/025.html')
regText = re.compile('текст страницы, которая находится выше\):(.*?)<p><b>Страницы учебника:', re.DOTALL)
if re.search(regText, page):
    text = re.search(regText, page).group(1)
text = re.sub(regSub,'', text)
c_text = re.sub(regTag, ' ', text)
if text.endswith('.'):
    c_text = c_text + '\n'

print(c_text)
#print(get_cleantext(mainpage))

 
 Пусть на развертке куба указаны отрезки OA, OB и АВ, где точки А я В — середины сторон соответствующих квадратов (рис. 17, а). Тогда соответствующие отрезки на гранях куба будут изображаться так, как показано на рисунке 17, б, причем точки А и В — середины отрезков, изображающих ребра куба. 
 Заметим, что при одном изображении куба (рис. 17, б) отрезок OA изображается штриховой линией. При другом изображении куба все три отрезка могут изображаться сплошной линией (рис. 17, в). 
 Задачи к § 3 1. На рисунке 18, а изображена развертка куба, точки Е и F — середины сторон квадрата. На ри- 
 



In [110]:
#test
with open('file.txt', 'r', encoding='utf-8') as b:
    tbook = b.read()
regText = re.compile('текст страницы, которая находится выше\):(.*?)<hr>', re.DOTALL)
regSub = re.compile('<sub>.*?</sub>', re.DOTALL)
regSup = re.compile('<sup>.*?</sup>', re.DOTALL)
regUnder = re.compile('<p><sub>.*?</sub>.*?\.</p>', re.DOTALL)
tbook = re.search(regText, tbook).group(1)
tbook = re.sub(regUnder,'', tbook)
tbook = re.sub(regSub,'', tbook)
#tbook = re.sub(regTag,'', tbook)

tbook

'</h1>\n<p>Глава I Начальные геометрические сведения</p>\n<p>Прямая и отрезок</p>\n<p>1 Точки, прямые, отрезки</p>\n<p>Вспомним, что нам известно о точках и прямых. Мы знаем, что для изображения прямых на чертеже пользуются линейкой (рис. 4), но при этом можно изобразить лишь часть прямой, а всю прямую мы представляем себе простирающейся бесконечно в обе стороны.</p>\n<p>Обычно прямые обозначают малыми латинскими буквами, а точки— большими латинскими буквами. На рисунке 5 изображены прямая а и точки А, Б, С и D. Точки А и В лежат на прямой а, а точки С и D не лежат на этой прямой. Можно сказать, что прямая а проходит через точки А и Б, но не проходит через точки С и D. Отметим, что через точки А и Б нельзя прове :ти другую прямую, не совпадающую с прям&lt; й а. Вообще,</p>\n<p>/</p>\n<p>Рис. 4</p>\n<p>Рис. 5</p>\n<p>через любые две точки можно провести прямую, и притом только одну<sup>1</sup>.</p>\n<p>Рассмотрим теперь две прямые.</p>\n<p>Если они имеют общую точку, то говорят, что эти

In [20]:
def get_numbofpages(page): ## getting number of html pages for a book
    regPages = re.compile('Страницы учебника:.*?(\d+)</A> </div></p>', re.DOTALL)
    if re.search(regPages, page):
        total = re.search(regPages, page).group(1)
    return total

In [89]:
def get_cleantext(page): ## getting clean text from the page
    regText = re.compile('текст страницы, которая находится выше\):(.*?)<hr>', re.DOTALL)
    regSub = re.compile('<sub>.*?</sub>', re.DOTALL)
    regSup = re.compile('<sup>.*?</sup>', re.DOTALL)
    regUnder = re.compile('<p><sub>.*?</sub>.*?\.</p>', re.DOTALL)
    regTag = regTag = re.compile('<.*?>', re.DOTALL)
    if re.search(regText, page):
        text = re.search(regText, page).group(1)
    text = re.sub(regSub,'', text)
    text = re.sub(regUnder,'', text)
    text = re.sub(regSup,'', text)
    text = re.sub(regTag, ' ', text)
    text = re.sub(r'&.*?;', '', text)
    text = re.sub(r'[Рр]ис\. \d+? ', '', text)
    text = re.sub(r'(\n)+', '\n', text)
    text = re.sub(r'(\s)+', ' ', text)
    return text

In [104]:
def get_books(): ## getting all geometry books from the site
    for idx, link in enumerate(links):
        main_link = common_link + link
        mainpage = download_html(main_link)
        total = get_numbofpages(mainpage)
        for i in range(5, int(total)):
            print(idx, i)
            if len(str(i)) == 1:
                current = download_html(main_link + '00' + str(i) + '.html')
            elif len(str(i)) == 2:
                current = download_html(main_link + '0' + str(i) + '.html')
            else:
                current = download_html(main_link + str(i) + '.html')
            curr_text = get_cleantext(current)
            with open('geometry/book' + str(idx) + '.txt', 'a', encoding = 'utf-8') as b:
                b.write(curr_text)

In [64]:
get_books() # got all books

0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
0 120
0 121
0 122
0 123
0 124
0 125
0 126
0 127
0 128
0 129
0 130
0 131
0 132
0 133
0 134
0 135
0 136
0 137
0 138
0 139
0 140
0 141
0 142
0 143
0 144
0 145
0 146
0 147
0 148
0 149
0 150
0 151
0 152
0 153
0 154
0 155
0 156
0 157
0 158
0 159
0 160
0 161
0 162
0 163
0 164
0 165
0 166
0 167
0 168
0 169
0 170
0 171
0 172
0 173
0 174
0 175
0 176
0 177
0 178
0 179
0 180
0 181
0 182
0 183
0 184
0 185
0 186
0 187
0 

4 283
4 284
4 285
4 286
4 287
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 16
5 17
5 18
5 19
5 20
5 21
5 22
5 23
5 24
5 25
5 26
5 27
5 28
5 29
5 30
5 31
5 32
5 33
5 34
5 35
5 36
5 37
5 38
5 39
5 40
5 41
5 42
5 43
5 44
5 45
5 46
5 47
5 48
5 49
5 50
5 51
5 52
5 53
5 54
5 55
5 56
5 57
5 58
5 59
5 60
5 61
5 62
5 63
5 64
5 65
5 66
5 67
5 68
5 69
5 70
5 71
5 72
5 73
5 74
5 75
5 76
5 77
5 78
5 79
5 80
5 81
5 82
5 83
5 84
5 85
5 86
5 87
5 88
5 89
5 90
5 91
5 92
5 93
5 94
5 95
5 96
5 97
5 98
5 99
5 100
5 101
5 102
5 103
5 104
5 105
5 106
5 107
5 108
5 109
5 110
5 111
5 112
5 113
5 114
5 115
5 116
5 117
5 118
5 119
5 120
5 121
5 122
5 123
5 124
5 125
5 126
5 127
5 128
5 129
5 130
5 131
5 132
5 133
5 134
5 135
5 136
5 137
5 138
5 139
5 140
5 141
5 142
5 143
5 144
5 145
5 146
5 147
5 148
5 149
5 150
5 151
5 152
5 153
5 154
5 155
5 156
5 157
5 158
5 159
5 160
5 161
5 162
5 163
5 164
5 165
5 166
5 167
5 168
5 169
5 170
5 171
5 172
5 173
5 174
5 175
5 176
5 177
5 178
5 179
5 180
5 181
5 182
5 

In [66]:
from gensim.utils import tokenize
from gensim.summarization.textcleaner import split_sentences

C:\Users\Sony\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [15]:
## cleaning the book which was collected manually
with open('book8.txt', 'r', encoding='utf-8') as b:
    book = b.read()

regQL = re.compile('\?[A-Za-z]+')
regLt = re.compile(' [A-Za-z]+')
regOp = re.compile('[\\=+*°<>/]')
book = re.sub(regOp, '', book)
book = re.sub(r'\d\)', '', book)
book = re.sub(r'\d\.', '', book)
book = re.sub(r'\d', '', book)
book = re.sub(regQL, '', book)
book = re.sub(regLt, '', book)
book = re.sub(r'\?{2,}', '', book)
book = re.sub(r' +', ' ', book)
book = re.sub(r'(\n|\n )+', '\n', book)

with open('new_book8.txt', 'w', encoding='utf-8') as b:
    b.write(book)

In [54]:
#got more o less clean texts for the first 8 books by changing the book titles in "with open(title, ...)"

with open('book7.txt', 'r', encoding='utf-8') as b:
    book = b.read()

regPic = re.compile('(Рис\.|рис\.)')
regBrPic = re.compile('\(.*?рис\..*?\)')
regLt = re.compile(' [A-Za-z]+')
regOp = re.compile('[\\=+*°<>/\|]')
book = re.sub(regOp, '', book)
book = re.sub(r'&.*?;', '', book)
book = re.sub(r'\d\)', '', book)
book = re.sub(r'\d\.', '', book)
book = re.sub(r'§ ', '', book)
book = re.sub(r'\d', '', book)
book = re.sub(regBrPic, '', book)
book = re.sub(regPic, '', book)
book = re.sub(r' \( ', '', book)
book = re.sub(regLt, '', book)
book = re.sub(r' +', ' ', book)
book = re.sub(r'(\n|\n )+', '\n', book)

with open('new_book7.txt', 'w', encoding='utf-8') as b:
    b.write(book)

In [56]:
#getting all books in one file
for root, dirs, files in os.walk('geometry'):
    for name in files:
        with open(os.path.join('geometry', name), 'r', encoding='utf-8') as b:
            book = b.read()
        print(name, len(book))
        with open('all geometry.txt', 'a', encoding = 'utf-8') as b:
            b.write(book)

with open('all geometry.txt', 'r', encoding = 'utf-8') as b:
    allbook = b.read()
print(len(allbook))

new_book0.txt 171389
new_book1.txt 497556
new_book2.txt 301494
new_book3.txt 469244
new_book4.txt 438994
new_book5.txt 409571
new_book6.txt 665162
new_book7.txt 410437
new_book8.txt 100120
3463967
